In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('D:\\Dropbox\\stempy')
sys.path.append('D:\\Dropbox\\stemplot')
    
from stemplot import *

from stempy.io import *
from stempy.clustering import *
from stempy.plot import *
from stempy.denoise import *
from stempy.utils import *
from stempy.feature import *
from stempy.spatial import *
from stempy.manifold import *

## Find points from HAADF

In [2]:
dp = r'D:\Dropbox\data\Leyi\Dopant Valency Project\CoWSe2_total 9 datasets\CoWSe2_20211124'+'\\'

num = 9

file_name1 = dp + 'ADF1_{}.dm4'.format(num)

# load image
img = load_image(file_name1)

img = normalize_image(img, 0, 1)

In [3]:
img1 = remove_bg(img, 11)

imshow(img1)

In [4]:
get_window_size(img)

13

## denoising

In [5]:
n_components = 32
patch_size = 32
extraction_step = 4
imgf = denoise_svd(img1, n_components, patch_size, extraction_step) 

Extracting reference patches...
done in 0.30s.
Singular value decomposition...
done in 2.08s.
Reconstructing...
done in 1.17s.


In [6]:
imshow(imgf)

In [7]:
threshold = 0.3
pts = local_max(imgf, min_distance=1, threshold=threshold)

## Clustering using MAADF

In [8]:
file_name2 = dp + 'ADF2_{}.dm4'.format(num)

# load image
img = load_image(file_name2)

img = normalize_image(img, 0, 1)

img2 = remove_bg(img, 11)

In [9]:
imshow(img2)

In [10]:
size = 26
kp = KeyPoints(pts, img2, size)
ps = kp.extract_patches(size) 

In [11]:
imshow(ps)

In [12]:
zps = ZPs(n_max=10, size=ps.shape[1])
zps.fit(ps)
X = zps.moments
X_rot = X.rotinv() 

In [13]:
plot_pca(X, 2)

In [14]:
fg = ForceGraph8(X = X,
n_neighbors=10,
init_mode = 'pca',
num_iterations = 100,
num_negative_samples=10,
force_params1=(0, 2, 1, 1),
force_params2=(2, 5, 5, 1),
divide=0.5)

from time import time
t0 = time()
xy = fg.fit_transform(X)

Construct graph from data...
Initialize 2-d embedding using PCA layout...


In [17]:
l= interactive_clusters(xy, img, kp.pts, ps, clip=False, s=1) 

In [16]:
len(l.ind), l.ind

(104,
 array([   76,   207,   238,   250,   296,   403,   404,   616,   697,
          899,   908,   998,  1006,  1163,  1173,  1437,  1492,  1499,
         1500,  1554,  1760,  2083,  2255,  2311,  2443,  2465,  2471,
         2527,  2909,  2933,  3005,  3307,  3332,  3337,  3338,  3438,
         3797,  3851,  4048,  4100,  4125,  4131,  4144,  4273,  4304,
         4414,  4443,  4473,  4489,  4524,  4634,  4721,  5020,  5227,
         5257,  5444,  5603,  5683,  5785,  5820,  6048,  6052,  6073,
         6102,  6132,  6215,  6238,  6334,  6369,  6391,  6594,  6727,
         6742,  6988,  7416,  7511,  7677,  7723,  7772,  7845,  7883,
         8174,  8260,  8274,  8276,  8313,  8326,  8378,  8592,  8611,
         8656,  8920,  8966,  9131,  9261,  9262,  9733,  9737,  9738,
        10017, 10056, 10074, 10274, 10430], dtype=int64))

In [16]:
kp.pts

array([[ 48,   8],
       [ 78,   8],
       [121,   8],
       ...,
       [452, 504],
       [461, 504],
       [482, 504]], dtype=int64)

In [17]:
np.save('pts_Ti_25.npy', kp.pts)

In [18]:
np.save('ind_Ti_25.npy', l.ind)